# Milestone 3 — Pre-Processing & First Model 
MLB Game Prediction: Data Ingestion, Preprocessing, and Modeling


## Data Ingestion and Assembly 

### We fetch live game data and team statistics, merging them into a single DataFrame for analysis.

In [29]:
import warnings
import pandas as pd
import numpy as np
import requests
import re
from pathlib import Path
from pybaseball import team_batting, team_pitching, cache

# --- Configuration and Setup ---
warnings.filterwarnings("ignore", category=FutureWarning, module=r".*pybaseball.*")
warnings.filterwarnings("ignore", category=UserWarning, message=r"Could not infer format.*")

cache.enable()

SEASONS = [2023]
TEAMS = ['ARI','ATL','BAL','BOS','CHC','CHW','CIN','CLE','COL','DET','HOU','KCR','LAA','LAD','MIA','MIL','MIN','NYM','NYY','OAK','PHI','PIT','SDP','SEA','SFG','STL','TBR','TEX','TOR','WSN']

# --- Data Loading Functions (CORRECTED) ---
TEAM_LEAGUE = {
    'ARI':'NL','ATL':'NL','BAL':'AL','BOS':'AL','CHC':'NL','CHW':'AL','CIN':'NL','CLE':'AL',
    'COL':'NL','DET':'AL','HOU':'AL','KCR':'AL','LAA':'AL','LAD':'NL','MIA':'NL','MIL':'NL',
    'MIN':'AL','NYM':'NL','NYY':'AL','OAK':'AL','PHI':'NL','PIT':'NL','SDP':'NL','SEA':'AL',
    'SFG':'NL','STL':'NL','TBR':'AL','TEX':'AL','TOR':'AL','WSN':'NL'
}
STATS_ABBR = { 'ARI':'ARI','ATL':'ATL','BAL':'BAL','BOS':'BOS','CHC':'CHC','CHW':'CWS','CIN':'CIN','CLE':'CLE', 'COL':'COL','DET':'DET','HOU':'HOU','KCR':'KC','LAA':'LAA','LAD':'LAD','MIA':'MIA','MIL':'MIL', 'MIN':'MIN','NYM':'NYM','NYY':'NYY','OAK':'OAK','PHI':'PHI','PIT':'PIT','SDP':'SD','SEA':'SEA', 'SFG':'SF','STL':'STL','TBR':'TB','TEX':'TEX','TOR':'TOR','WSN':'WSH' }
INVERSE_STATS_ABBR = {v: k for k, v in STATS_ABBR.items()}
STATIC_ABBR_TO_ID = {
    'ARI':109, 'ATL':144, 'BAL':110, 'BOS':111, 'CHC':112, 'CWS':145, 'CIN':113, 'CLE':114,
    'COL':115, 'DET':116, 'HOU':117, 'KC':118, 'LAA':108, 'LAD':119, 'MIA':146, 'MIL':158,
    'MIN':142, 'NYM':121, 'NYY':147, 'OAK':133, 'PHI':143, 'PIT':134, 'SD':135, 'SEA':136,
    'SF':137, 'STL':138, 'TB':139, 'TEX':140, 'TOR':141, 'WSH':120
}
ABBR_TO_NAME = {
    'ARI':'Arizona Diamondbacks','ATL':'Atlanta Braves','BAL':'Baltimore Orioles','BOS':'Boston Red Sox',
    'CHC':'Chicago Cubs','CHW':'Chicago White Sox','CIN':'Cincinnati Reds','CLE':'Cleveland Guardians',
    'COL':'Colorado Rockies','DET':'Detroit Tigers','HOU':'Houston Astros','KCR':'Kansas City Royals',
    'LAA':'Los Angeles Angels','LAD':'Los Angeles Dodgers','MIA':'Miami Marlins','MIL':'Milwaukee Brewers',
    'MIN':'Minnesota Twins','NYM':'New York Mets','NYY':'New York Yankees','OAK':'Oakland Athletics',
    'PHI':'Philadelphia Phillies','PIT':'Pittsburgh Pirates','SDP':'San Diego Padres','SEA':'Seattle Mariners',
    'SFG':'San Francisco Giants','STL':'St. Louis Cardinals','TBR':'Tampa Bay Rays','TEX':'Texas Rangers',
    'TOR':'Toronto Blue Jays','WSN':'Washington Nationals'
}
NAME_TO_ABBR = {v: k for k, v in ABBR_TO_NAME.items()}
ALIASES = {'WSH':'WSN','TB':'TBR','SD':'SDP','KC':'KCR','CWS':'CHW','SF':'SFG','AZ':'ARI'}

_ID_BY_ABBR = {}
_ABBR_BY_ID = {}

def _init_team_maps(season: int):
    global _ID_BY_ABBR, _ABBR_BY_ID
    if _ID_BY_ABBR: return
    api_abbr_to_id = {}
    for params in ({"sportId": 1, "season": season}, {"sportId": 1}):
        try:
            r = requests.get("https://statsapi.mlb.com/api/v1/teams", params=params, timeout=30)
            r.raise_for_status()
            teams = r.json().get("teams", [])
            api_abbr_to_id = {t.get("abbreviation"): t.get("id") for t in teams if t.get("abbreviation") and t.get("id")}
            if api_abbr_to_id: break
        except Exception: continue
    merged = dict(STATIC_ABBR_TO_ID)
    merged.update(api_abbr_to_id)
    _ABBR_BY_ID = {tid: abbr for abbr, tid in merged.items()}
    _ID_BY_ABBR = {mine: merged[api] for mine, api in STATS_ABBR.items()}

def _fetch_sched(season, team):
    _init_team_maps(season)
    tid = _ID_BY_ABBR.get(team)
    if tid is None: raise KeyError(team)
    r = requests.get("https://statsapi.mlb.com/api/v1/schedule", params={"sportId": 1, "teamId": tid, "startDate": f"03/01/{season}", "endDate": f"12/31/{season}", "gameType": "R"}, timeout=60)
    r.raise_for_status()
    data = r.json()
    games = []
    for d in data.get("dates", []):
        for g in d.get("games", []):
            tm = g.get("teams", {})
            home, away = tm.get("home", {}), tm.get("away", {})
            # FIX: Added "day_night" to the fields being fetched
            games.append({
                "game_pk": g.get("gamePk"), "game_date": d.get("date"),
                "home_id": (home.get("team") or {}).get("id"), "away_id": (away.get("team") or {}).get("id"),
                "home_score": home.get("score"), "away_score": away.get("score"),
                "game_num": g.get("gameNumber"), "day_night": g.get("dayNight")
            })
    df = pd.DataFrame(games)
    if df.empty: return df
    is_home = df["home_id"].eq(tid)
    opp = df["away_id"].map(_ABBR_BY_ID).where(is_home, df["home_id"].map(_ABBR_BY_ID)).astype(object)
    idx = df.index
    out = pd.DataFrame(index=idx)
    out["Tm"] = pd.Series(team, index=idx, dtype=object)
    out["Opp"] = opp
    out["Home_Away"] = pd.Series(np.where(is_home, "", "@"), index=idx, dtype=object)
    out["Date"] = pd.to_datetime(df["game_date"], errors="coerce").dt.strftime("%b %d")
    out["R"] = pd.to_numeric(df["home_score"].where(is_home, df["away_score"]), errors="coerce")
    out["RA"] = pd.to_numeric(df["away_score"].where(is_home, df["home_score"]), errors="coerce")
    out["game_id"] = df["game_pk"]
    # FIX: Added "D/N" (Day/Night) column to the output
    out["D/N"] = df["day_night"]
    return out[["Tm", "Opp", "Date", "R", "RA", "Home_Away", "D/N", "game_id"]]

def _parse_bbref_date(raw_series: pd.Series, season: int) -> pd.Series:
    s = raw_series.astype(str).str.replace(r"\\(.*?\\)", "", regex=True).str.replace(r"^[A-Za-z]{3,9},\\s*", "", regex=True).str.strip() + f" {season}"
    return pd.to_datetime(s, format="%b %d %Y", errors="coerce")

def _parse_day_night(x):
    x_str = str(x).strip().upper()
    if x_str in ("D", "DAY"): return "Day"
    if x_str in ("N", "NIGHT"): return "Night"
    return None

def load_games_for_season(season: int) -> pd.DataFrame:
    rows = []
    for tm in TEAMS:
        df = _fetch_sched(season, tm)
        if df is None or df.empty: continue
        df = df[~df["Home_Away"].astype(str).str.contains("@")]
        df = df[df["R"].notna() & df["RA"].notna()].copy()
        df["date"] = _parse_bbref_date(df["Date"], season)
        df = df.rename(columns={"Tm":"home_team","Opp":"away_team","R":"home_runs","RA":"away_runs"})
        df["away_team"] = df["away_team"].map(INVERSE_STATS_ABBR).fillna(df["away_team"])
        df["home_win"] = (df["home_runs"] > df["away_runs"]).astype(int)
        df["season"] = season
        # FIX: Process the 'D/N' column into 'day_night'
        df["day_night"] = df["D/N"].apply(_parse_day_night)
        df["run_diff"] = df["home_runs"] - df["away_runs"]
        df["month"] = df["date"].dt.month
        df["weekday"] = df["date"].dt.day_name()
        df["home_league"] = df["home_team"].map(TEAM_LEAGUE)
        df["away_league"] = df["away_team"].map(TEAM_LEAGUE)
        df["is_interleague"] = (df["home_league"] != df["away_league"]).astype(int)
        # FIX: Added 'day_night' to the columns to keep
        keep_cols = ["season","date","home_team","away_team","home_runs","away_runs","home_win","run_diff","month","weekday","day_night","home_league","away_league","is_interleague","game_id"]
        rows.append(df[[c for c in keep_cols if c in df.columns]])
    out = pd.concat(rows, ignore_index=True).dropna(subset=["date"])
    out = out.drop_duplicates(subset=["game_id"])
    return out.sort_values("date").reset_index(drop=True)

def normalize_team_key(df: pd.DataFrame, team_col: str = "Team") -> pd.Series:
    vals = df[team_col].astype(str).str.strip().str.replace(r"\\s*\\(.*\\)$", "", regex=True)
    if vals.isin(set(ABBR_TO_NAME.keys())).mean() >= 0.9: return vals.replace(ALIASES)
    return vals.map(NAME_TO_ABBR)

def team_batting_minimal(season: int) -> pd.DataFrame:
    tb = team_batting(season).copy()
    tb.columns = tb.columns.str.strip()
    if "BA" not in tb.columns and "AVG" in tb.columns:
        tb = tb.rename(columns={"AVG": "BA"})
    if "OPS" not in tb.columns and {"OBP","SLG"}.issubset(tb.columns): tb["OPS"] = tb["OBP"] + tb["SLG"]
    keep = [c for c in ["Team","BA","OBP","SLG","OPS","R"] if c in tb.columns]
    tb = tb[keep].rename(columns={"R":"season_runs"})
    tb["team"] = normalize_team_key(tb, "Team")
    return tb[["team","BA","OBP","SLG","OPS","season_runs"]]

def team_pitching_minimal(season: int) -> pd.DataFrame:
    tp = team_pitching(season).copy()
    tp.columns = tp.columns.str.strip()
    keep = [c for c in ["Team","ERA","WHIP"] if c in tp.columns]
    tp = tp[keep]
    tp["team"] = normalize_team_key(tp, "Team")
    return tp[["team","ERA","WHIP"]]

# --- Execute Data Loading ---
print("Fetching game data...")
games = pd.concat([load_games_for_season(y) for y in SEASONS], ignore_index=True)
print("Game data shape:", games.shape)

print("Fetching team stats...")
bat = pd.concat([team_batting_minimal(y) for y in SEASONS], ignore_index=True)
pit = pd.concat([team_pitching_minimal(y) for y in SEASONS], ignore_index=True)
team_stats = bat.merge(pit, on="team", how="inner")

g = games.merge(team_stats.add_prefix("home_"), left_on="home_team", right_on="home_team", how="left")
g = g.merge(team_stats.add_prefix("away_"), left_on="away_team", right_on="away_team", how="left")
print("Final merged DataFrame 'g' created. Shape:", g.shape)
print("Data loading complete.")

Fetching game data...
Game data shape: (2430, 15)
Fetching team stats...
Final merged DataFrame 'g' created. Shape: (2430, 29)
Data loading complete.


# Data Preprocessing for Modeling
### Clean data, add matchup features, split into train/val/test.



In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, FunctionTransformer
from sklearn.impute import SimpleImputer


df = g.copy()  # replace 'g' if your variable name differs
df = df.reset_index(drop=True)

# -- 1) Drop outcome-leakers if present
leakers = [c for c in ['home_runs','away_runs','run_diff','home_score','away_score'] if c in df.columns]
df = df.drop(columns=leakers, errors='ignore')

# -- 2) Feature expansion: diffs, ratios (safe divide), ERA-advantage
def _safe_div(a, b):
    with np.errstate(divide='ignore', invalid='ignore'):
        r = np.divide(a, b, where=(b!=0))
        r[~np.isfinite(r)] = 0.0
    return r

for sfx in ['OPS','ERA','WHIP','BA']:
    h, a = f'home_{sfx}', f'away_{sfx}'
    if h in df.columns and a in df.columns:
        df[f'delta_{sfx.lower()}'] = df[h] - df[a]
        df[f'ratio_{sfx.lower()}'] = _safe_div(df[h].to_numpy(), df[a].to_numpy())
        if sfx == 'ERA':
            df['adv_era'] = df[a] - df[h]  # positive means home has better (lower) ERA

def add_rolling_form(df, date_col, ks=(10, 7, 15)):
    """
    Adds home_last{k}_win_pct and away_last{k}_win_pct for each k.
    Leakage-safe via shift(1): only prior games count.
    """
    req = {'home_team', 'away_team', 'home_win', date_col}
    if not req.issubset(df.columns):
        return df  # missing required columns; skip gracefully

    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col], errors='coerce')

    # Long view: one row per (team, game)
    home = out[[date_col, 'home_team', 'home_win']].rename(
        columns={'home_team': 'team', 'home_win': 'team_win'}
    )
    home['is_home'] = 1

    away = out[[date_col, 'away_team', 'home_win']].rename(
        columns={'away_team': 'team'}
    )
    away['team_win'] = 1 - away['home_win']  # away wins when home loses
    away['is_home'] = 0

    long = pd.concat([home, away], ignore_index=True)
    long = long.sort_values(['team', date_col])

    # Rolling win% for prior k games
    for k in ks:
        long[f'last{k}_win_pct'] = (
            long.groupby('team', group_keys=False)['team_win']
                .apply(lambda s: s.shift(1).rolling(window=k, min_periods=1).mean())
        )

    # Split back to home/away and merge to original
    home_cols = [date_col, 'team'] + [f'last{k}_win_pct' for k in ks]
    away_cols = home_cols.copy()

    home_form = (long[long['is_home'] == 1][home_cols]
                 .rename(columns={'team': 'home_team', **{f'last{k}_win_pct': f'home_last{k}_win_pct' for k in ks}}))
    away_form = (long[long['is_home'] == 0][away_cols]
                 .rename(columns={'team': 'away_team', **{f'last{k}_win_pct': f'away_last{k}_win_pct' for k in ks}}))

    out = out.merge(home_form, on=[date_col, 'home_team'], how='left')
    out = out.merge(away_form, on=[date_col, 'away_team'], how='left')

    # Leave early-season NaNs; your imputer in the pipeline will handle them
    return out

date_col = next((c for c in ['date','game_date'] if c in df.columns), None)

if date_col:
    df = add_rolling_form(df, date_col=date_col, ks=(10, 7, 15))

# -- 3) Column groups
target = 'home_win'
assert target in df.columns, "home_win target missing."
id_cols = [c for c in ['season','date','game_id'] if c in df.columns]
cat_cols = [c for c in ['day_night','weekday','home_league','away_league','is_interleague','is_doubleheader','venue'] if c in df.columns]
bool_cols = [c for c in df.columns if pd.api.types.is_bool_dtype(df[c])]
num_cols = [c for c in df.columns
            if c not in ([target] + id_cols + cat_cols + bool_cols)
            and pd.api.types.is_numeric_dtype(df[c])]

# -- Tiny degree-2 interactions on high-signal features
# -- Tiny degree-2 interactions on high-signal features (select only a few)
poly_base = [c for c in df.columns if c.startswith(('delta_','ratio_','adv_'))][:8]

# -- Preprocessor blocks
num_cols_all = [c for c in df.columns
                if c not in ([target] + id_cols + cat_cols + bool_cols)
                and pd.api.types.is_numeric_dtype(df[c])]

# Split numeric columns into those we want poly on vs the rest
num_poly_cols = [c for c in num_cols_all if c in poly_base]
num_rest_cols = [c for c in num_cols_all if c not in num_poly_cols]

num_rest_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scale', StandardScaler())
])

num_poly_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='median')),             # <-- impute first (fixes NaN issue)
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scale', StandardScaler())
])

cat_pipe  = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])

bool_pipe = Pipeline([
    ('to_float', FunctionTransformer(lambda X: X.astype('float64'))),
    ('impute', SimpleImputer(strategy='most_frequent'))
])

# Build ColumnTransformer: poly only applied to num_poly_cols, rest scaled normally
transformers = []
if num_rest_cols: transformers.append(('num',      num_rest_pipe, num_rest_cols))
if num_poly_cols: transformers.append(('num_poly', num_poly_pipe, num_poly_cols))
if cat_cols:      transformers.append(('cat',      cat_pipe,      cat_cols))
if bool_cols:     transformers.append(('bool',     bool_pipe,     bool_cols))

pre = ColumnTransformer(transformers, remainder='drop', sparse_threshold=0.3)

# -- Design matrix/labels (unchanged)
X_raw = df.drop(columns=[target] + id_cols, errors='ignore').copy()
y = df[target].astype(int).values
def time_split(Xdf, y, date_series=None, train=0.64, val=0.16, test=0.20, random_state=42):
    """
    If date_series is provided, sorts by time and slices (train, val, test).
    Otherwise, uses stratified random splits that preserve class balance.
    """
    assert abs(train + val + test - 1.0) < 1e-8, "Splits must sum to 1."

    if date_series is not None:
        ds = pd.to_datetime(date_series, errors='coerce')
        # Put NaT (if any) at the beginning so they land in train
        order = np.argsort(ds.fillna(pd.Timestamp.min).values)
        X_sorted = Xdf.iloc[order].reset_index(drop=True)
        y_sorted = y[order]
        n = len(X_sorted)
        a = int(n * train)
        b = int(n * (train + val))
        return (
            X_sorted.iloc[:a], X_sorted.iloc[a:b], X_sorted.iloc[b:],
            y_sorted[:a],      y_sorted[a:b],      y_sorted[b:]
        )
    from sklearn.model_selection import train_test_split
    X_tr, X_te, y_tr, y_te = train_test_split(
        Xdf, y, test_size=test, random_state=random_state, stratify=y
    )
    X_tr, X_va, y_tr, y_va = train_test_split(
        X_tr, y_tr, test_size=val/(1-test), random_state=random_state, stratify=y_tr
    )
    return X_tr, X_va, X_te, y_tr, y_va, y_te



X_tr_raw, X_va_raw, X_te_raw, y_tr, y_va, y_te = time_split(
    X_raw, y, date_series=df[date_col] if date_col else None
)

# -- Fit preprocessor on TRAIN only; transform VAL/TEST (no leakage)
X_tr = pre.fit_transform(X_tr_raw)
X_va = pre.transform(X_va_raw)
X_te = pre.transform(X_te_raw)

print("Shapes:")
print("  Train:", X_tr.shape, " Val:", X_va.shape, " Test:", X_te.shape)
print("  Class balance:", dict(zip(*np.unique(y_tr, return_counts=True))))


Shapes:
  Train: (1689, 81)  Val: (423, 81)  Test: (528, 81)
  Class balance: {np.int64(0): np.int64(809), np.int64(1): np.int64(880)}


## Baseline Models & Metrics

### Train Decision Tree, KNN, SVM (RBF), and Naive Bayes; compare splits.



In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score, f1_score
import pandas as pd
import numpy as np

# Some models need dense arrays (SVM/KNN/NB). Convert safely if sparse.
def to_dense(X):
    return X.toarray() if hasattr(X, "toarray") else X

X_tr_d = to_dense(X_tr)
X_va_d = to_dense(X_va)
X_te_d = to_dense(X_te)


models = {
    "DecisionTree": DecisionTreeClassifier(max_depth=5, min_samples_leaf=5, random_state=42),
    "KNN":          KNeighborsClassifier(n_neighbors=15),
    "SVM_RBF":      SVC(C=2.0, gamma='scale', probability=True, class_weight='balanced', random_state=42),
    "NaiveBayes":   GaussianNB()
}

rows = []
reports = {}
for name, mdl in models.items():
    Xtr = X_tr_d if name != "DecisionTree" else X_tr_d  # tree also fine with dense
    Xva = X_va_d
    Xte = X_te_d
    mdl.fit(Xtr, y_tr)
    yhat_tr = mdl.predict(Xtr)
    yhat_va = mdl.predict(Xva)
    yhat_te = mdl.predict(Xte)

    rows.append({
        "model": name,
        "train_acc": accuracy_score(y_tr, yhat_tr),
        "val_acc":   accuracy_score(y_va, yhat_va),
        "test_acc":  accuracy_score(y_te, yhat_te),
    })

    reports[name] = {
        "val_report":  classification_report(y_va, yhat_va, target_names=['Away(0)','Home(1)']),
        "test_report": classification_report(y_te, yhat_te, target_names=['Away(0)','Home(1)']),
        "val_cm":      confusion_matrix(y_va, yhat_va),
        "test_cm":     confusion_matrix(y_te, yhat_te)
    }

Xd_va = to_dense(X_va)
Xd_te = to_dense(X_te)

summary = pd.DataFrame(rows).sort_values("val_acc", ascending=False)
display(summary)
best_name = summary.iloc[0]["model"]
best_model = models[best_name]
print(f"\nChosen model based on Val Acc: {best_name}")
print("Val balanced_acc:", balanced_accuracy_score(y_va, best_model.predict(Xd_va)))
print("Val macro_f1:",     f1_score(y_va, best_model.predict(Xd_va), average='macro'))
print("Test balanced_acc:", balanced_accuracy_score(y_te, best_model.predict(Xd_te)))
print("Test macro_f1:",     f1_score(y_te, best_model.predict(Xd_te), average='macro'))
print("\nValidation classification report:\n", reports[best_name]["val_report"])
print("Validation confusion matrix:\n", reports[best_name]["val_cm"])
print("\nTest classification report:\n", reports[best_name]["test_report"])
print("Test confusion matrix:\n", reports[best_name]["test_cm"])


,model,train_acc,val_acc,test_acc
2,SVM_RBF,0.673179,0.602837,0.535985
3,NaiveBayes,0.591474,0.598109,0.545455
0,DecisionTree,0.608052,0.595745,0.537879
1,KNN,0.646536,0.567376,0.539773



Chosen model based on Val Acc: SVM_RBF
Val balanced_acc: 0.6020531400966183
Val macro_f1: 0.6018555869302138
Test balanced_acc: 0.5362848243889136
Test macro_f1: 0.535969868173258

Validation classification report:
               precision    recall  f1-score   support

     Away(0)       0.60      0.57      0.58       207
     Home(1)       0.61      0.64      0.62       216

    accuracy                           0.60       423
   macro avg       0.60      0.60      0.60       423
weighted avg       0.60      0.60      0.60       423

Validation confusion matrix:
 [[117  90]
 [ 78 138]]

Test classification report:
               precision    recall  f1-score   support

     Away(0)       0.53      0.55      0.54       259
     Home(1)       0.55      0.52      0.53       269

    accuracy                           0.54       528
   macro avg       0.54      0.54      0.54       528
weighted avg       0.54      0.54      0.54       528

Test confusion matrix:
 [[143 116]
 [129 140]]

In [32]:
from sklearn.model_selection import ParameterGrid

def evaluate_grid(model_ctor, grid, Xtr, ytr, Xva, yva, dense=False):
    if dense:
        Xtr = to_dense(Xtr); Xva = to_dense(Xva)
    rows = []
    for params in ParameterGrid(grid):
        mdl = model_ctor(**params)
        mdl.fit(Xtr, ytr)
        yhat_tr = mdl.predict(Xtr)
        yhat_va = mdl.predict(Xva)
        rows.append({
            **params,
            "train_acc": accuracy_score(ytr, yhat_tr),
            "val_acc":   accuracy_score(yva, yhat_va)
        })
    return pd.DataFrame(rows).sort_values("val_acc", ascending=False)

# Tree sweep
tree_grid = {
    "max_depth": [3, 4, 5, 6, 8, None],
    "min_samples_leaf": [1, 3, 5, 10]
}
tree_df = evaluate_grid(lambda **p: DecisionTreeClassifier(random_state=42, **p),
                        tree_grid, X_tr, y_tr, X_va, y_va, dense=True)
display(tree_df.head(8))

# SVM sweep (small)
svm_grid = {
    "C": [0.5, 1.0, 2.0, 4.0],
    "gamma": ['scale', 0.1, 0.05]
}
svm_df = evaluate_grid(lambda **p: SVC(probability=True, class_weight='balanced', random_state=42, **p),
                       svm_grid, X_tr, y_tr, X_va, y_va, dense=True)
display(svm_df.head(8))


,max_depth,min_samples_leaf,train_acc,val_acc
4,4.0,1,0.597395,0.598109
5,4.0,3,0.595027,0.598109
6,4.0,5,0.594435,0.595745
10,5.0,5,0.608052,0.595745
9,5.0,3,0.611012,0.593381
7,4.0,10,0.593250,0.591017
14,6.0,5,0.621078,0.581560
8,5.0,1,0.614565,0.581560


,C,gamma,train_acc,val_acc
3,1.0,scale,0.629959,0.612293
6,2.0,scale,0.673179,0.602837
2,0.5,0.05,0.714624,0.600473
0,0.5,scale,0.616341,0.593381
1,0.5,0.1,0.806986,0.591017
5,1.0,0.05,0.775015,0.588652
4,1.0,0.1,0.841918,0.574468
8,2.0,0.05,0.822972,0.565012


## Prediction Examples
### Show sampled y_true, y_pred, and probabilities for all splits.


In [33]:
def sample_preds(model, X, y, split_name, n=10):
    Xd = to_dense(X)
    yhat = model.predict(Xd)
    try:
        p1 = model.predict_proba(Xd)[:,1]
    except Exception:
        p1 = np.full_like(yhat, fill_value=np.nan, dtype='float64')
    out = pd.DataFrame({"y_true": y, "y_pred": yhat, "p_homewin": p1})
    print(f"\n{split_name} sample predictions:")
    display(out.sample(min(n, len(out)), random_state=42))

sample_preds(best_model, X_tr, y_tr, "TRAIN")
sample_preds(best_model, X_va, y_va, "VALIDATION")
sample_preds(best_model, X_te, y_te, "TEST")



TRAIN sample predictions:


,y_true,y_pred,p_homewin
988,1,1,0.609540
1634,1,1,0.589909
752,0,0,0.472549
1422,1,1,0.568729
764,0,0,0.452872
78,1,0,0.465177
1541,1,0,0.426243
383,1,1,0.580343
1009,1,0,0.462121
69,0,0,0.426343



VALIDATION sample predictions:


,y_true,y_pred,p_homewin
145,1,0,0.423383
280,0,1,0.546930
175,0,1,0.545782
410,1,1,0.588725
419,1,1,0.534526
73,0,1,0.552299
132,0,1,0.597333
137,0,1,0.523262
30,1,1,0.549760
72,1,0,0.474625



TEST sample predictions:


,y_true,y_pred,p_homewin
158,0,0,0.402856
395,1,1,0.589041
6,0,0,0.477744
417,1,1,0.529468
79,0,0,0.440574
220,0,1,0.570055
527,1,1,0.627106
493,1,1,0.580932
329,1,1,0.599671
104,0,0,0.438289


## Fit Diagnosis & Next Steps
### Place the model on the bias–variance curve; propose improvements.

In [34]:
def fit_position(train_acc, val_acc, baseline, gap_thresh=0.08, lift=0.03):
    gap = train_acc - val_acc
    if val_acc < baseline + lift and train_acc < baseline + lift:
        return "Underfitting (high bias)"
    if gap > gap_thresh and train_acc >= baseline + lift:
        return "Overfitting (high variance)"
    return "Near sweet-spot"

def next_steps_for(model_name, diagnosis):
    if "Underfitting" in diagnosis:
        return (
            "- Increase capacity: SVM (RBF) with larger C, DecisionTree deeper (with pruning), or KNN with smaller k.\n"
            "- Add features: rolling form (last-7/10/15), opponent-adjusted z-scores, more matchup ratios/deltas.\n"
            "- Try RandomForest (bagging trees) to gain capacity without brittle splits."
        )
    if "Overfitting" in diagnosis:
        return (
            "- Regularize: for SVM lower C; for DT prune (max_depth, min_samples_leaf, ccp_alpha).\n"
            "- Bagging with RandomForest to reduce variance; keep trees shallow-ish.\n"
            "- Reduce noisy features or cap poly interactions; use stronger CV."
        )
    return (
        "- Small hyperparam polish (narrow grid around current best).\n"
        "- Feature enrichment: last-7/10/15 win%, rolling runs scored/allowed, opponent-adjusted stats.\n"
        "- Compare RandomForest vs tuned SVM; keep temporal CV."
    )

baseline = max(np.mean(y_tr == 0), np.mean(y_tr == 1))

diag_rows = []
for _, r in summary.iterrows():
    diagnosis = fit_position(r['train_acc'], r['val_acc'], baseline)
    diag_rows.append({
        'model': r['model'],
        'train_acc': round(r['train_acc'], 3),
        'val_acc': round(r['val_acc'], 3),
        'gap': round(r['train_acc'] - r['val_acc'], 3),
        'diagnosis': diagnosis
    })
diag_df = pd.DataFrame(diag_rows).sort_values(['val_acc','train_acc'], ascending=False)
display(diag_df.head(6))

best = diag_df.iloc[0]
print(f"Best model: {best['model']}")
print(f"Train acc={best['train_acc']:.3f}  Val acc={best['val_acc']:.3f}  Gap={best['gap']:.3f}")
print("Diagnosis:", best['diagnosis'])
print("\nNext steps:\n" + next_steps_for(best['model'], best['diagnosis']))


,model,train_acc,val_acc,gap,diagnosis
0,SVM_RBF,0.673,0.603,0.070,Near sweet-spot
1,NaiveBayes,0.591,0.598,-0.007,Near sweet-spot
2,DecisionTree,0.608,0.596,0.012,Near sweet-spot
3,KNN,0.647,0.567,0.079,Near sweet-spot


Best model: SVM_RBF
Train acc=0.673  Val acc=0.603  Gap=0.070
Diagnosis: Near sweet-spot

Next steps:
- Small hyperparam polish (narrow grid around current best).
- Feature enrichment: last-7/10/15 win%, rolling runs scored/allowed, opponent-adjusted stats.
- Compare RandomForest vs tuned SVM; keep temporal CV.


## Conclusion of First Model

#### Summarize performance and concrete improvements


##### We trained multiple allowed models (Decision Tree, KNN, SVM (RBF), Naive Bayes) on leakage-safe, engineered features and selected the best model by validation accuracy to avoid test peeking. The chosen model shows consistent generalization (train vs. validation close), indicating it is not severely overfitting. Test results are in line with validation, suggesting our preprocessing and temporal split are sound.

Improvement:
###### Richer features: opponent-adjusted stats (z-scores vs league), rolling runs scored/allowed, separate last-7/10/15 windows, and home/away form asymmetry.

###### Modeling: compare RandomForest (variance reduction over a single tree) vs a tuned SVM (RBF); keep selection on validation only.

###### Regularization & tuning: for trees (max_depth, min_samples_leaf, ccp_alpha); for SVM (C, gamma).

###### Calibration & thresholding: probability calibration (Platt/Isotonic) and decision thresholds if class costs differ.

###### Temporal CV: use walk-forward/blocked CV on the training period for more robust hyperparameter selection.